In [7]:
# Import libraries
import pandas as pd
import numpy as np
import gensim
import multiprocessing
from sklearn import utils
from ast import literal_eval

# Gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.models import KeyedVectors

# Progress bar
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [8]:
# Import dataset
df = pd.read_csv('../Data/4. Labeling/labeling.csv')

# Mengonversi kolom tokenisasi menjadi list
df['text_tokenize'] = df['text_tokenize'].apply(literal_eval)

text_tokenize = df['text_tokenize']

# Menampilkan total dataset
print("Total Dataset :", len(df))

df.head()

Total Dataset : 4776


,video_id,comment_text,text_cleaned,text_removal,text_normalized,text_manual_replaced,text_tokenize,label
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,"[baik, bagaimana, turut, kalian]",0
1,video1,Betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,"[betul, dok, betul, dok]",0
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di,betul dok mungkin efek sa,betul dok mungkin efek sa,betul dok mungkin efek sa,"[betul, dok, mungkin, efek, sa]",0
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu,bpom aneh kerja ko itu,bpom aneh kerja kok itu,bpom aneh kerja kok itu,"[bpom, aneh, kerja, kok, itu]",-1
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obat d...,benar banget anak saya saja harus sirup minum ...,benar banget anak aku saja harus sirup minum o...,"[benar, banget, anak, aku, saja, harus, sirup,...",0


TaggedDocument

In [9]:
def labelize_comments_ug(comments, label):
    """
    Fungsi ini mengonversi setiap komentar YouTube menjadi objek TaggedDocument 
    yang terdiri dari kata-kata dalam komentar dan label/tag yang terkait dengan komentar tersebut.
    """
    result = []
    prefix = label
    for i, c in zip(comments.index, comments):
        result.append(TaggedDocument(c, [prefix + '_%s' % i]))
    return result

In [10]:
# Mengonversi semua komentar YouTube menjadi objek TaggedDocument
all_comments_w2v = labelize_comments_ug(text_tokenize, 'all')

Word2Vec

Word2Vec dengan arsitektur Continuous Bag of Words (CBOW)

In [11]:
# Jumlah core CPU yang tersedia
cores = multiprocessing.cpu_count()

# Membuat model Word2Vec dengan arsitektur CBOW
model_ug_cbow = Word2Vec(
    sg=0,  # Menggunakan CBOW (sg=0)
    vector_size=100,  # Ukuran vektor fitur
    negative=5,  # Jumlah negative sampling
    window=2,  # Jarak maksimum antara kata target dan kata-kata yang mengelilinginya
    min_count=1,  # Minimum jumlah kemunculan kata untuk dimasukkan ke dalam model
    workers=cores,  # Jumlah worker yang akan digunakan
    alpha=0.065,  # Learning rate awal
    min_alpha=0.065,  # Learning rate minimum
    hs=1,  # Menggunakan hierarchical softmax
)

# Membangun kosakata dari semua komentar yang telah dikonversi menjadi TaggedDocument
model_ug_cbow.build_vocab([x.words for x in tqdm(all_comments_w2v)])

100%|██████████| 4776/4776 [00:00<00:00, 1595412.23it/s]


In [12]:
%%time
# Melatih model Word2Vec dengan arsitektur CBOW sebanyak 10 epoch 
for epoch in range(30):
    # Mengacak urutan data untuk setiap epoch
    shuffled_data = utils.shuffle([x.words for x in tqdm(all_comments_w2v)])
    
    # Melatih model dengan data yang telah diacak
    model_ug_cbow.train(shuffled_data, total_examples=len(all_comments_w2v), epochs=1)
    
    # Mengurangi learning rate sebesar 0.002 setiap epoch
    model_ug_cbow.alpha -= 0.002
    
    # Menyesuaikan learning rate minimum agar sama dengan learning rate saat ini
    model_ug_cbow.min_alpha = model_ug_cbow.alpha


100%|██████████| 4776/4776 [00:00<00:00, 2376556.64it/s]


CPU times: total: 8.17 s
Wall time: 2.31 s


In [13]:
# Membuat model Word2Vec dengan arsitektur Skip-gram
model_ug_sg = Word2Vec(
    sg=1,  # Menggunakan Skip-gram (sg=1)
    vector_size=100,  # Ukuran vektor fitur
    negative=5,  # Jumlah negative sampling
    window=2,  # Jarak maksimum antara kata target dan kata-kata yang mengelilinginya
    min_count=1,  # Minimum jumlah kemunculan kata untuk dimasukkan ke dalam model
    workers=cores,  # Jumlah worker yang akan digunakan
    alpha=0.065,  # Learning rate awal
    min_alpha=0.065,  # Learning rate minimum
    hs=1,  # Menggunakan hierarchical softmax
)

# Membangun kosakata dari semua komentar yang telah dikonversi menjadi TaggedDocument
model_ug_sg.build_vocab([x.words for x in tqdm(all_comments_w2v)])


100%|██████████| 4776/4776 [00:00<00:00, 4779765.19it/s]


In [14]:
%%time
# Melatih model Word2Vec dengan arsitektur Skip-gram sebanyak 10 epoch
for epoch in range(30):
    # Mengacak urutan data untuk setiap epoch
    shuffled_data = utils.shuffle([x.words for x in tqdm(all_comments_w2v)])
    
    # Melatih model dengan data yang telah diacak
    model_ug_sg.train(shuffled_data, total_examples=len(all_comments_w2v), epochs=1)
    
    # Mengurangi learning rate sebesar 0.002 setiap epoch
    model_ug_sg.alpha -= 0.002
    
    # Menyesuaikan learning rate minimum agar sama dengan learning rate saat ini
    model_ug_sg.min_alpha = model_ug_sg.alpha


100%|██████████| 4776/4776 [00:00<00:00, 2394739.50it/s]


CPU times: total: 19.1 s
Wall time: 3.76 s


In [15]:
# Menyimpan model Word2Vec CBOW yang telah dilatih
model_ug_cbow.save('../Model/w2v_model_ug_cbow.word2vec')

# Menyimpan model Word2Vec Skip-gram yang telah dilatih
model_ug_sg.save('../Model/w2v_model_ug_sg.word2vec')